In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from konlpy.tag import Twitter

from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
from os import path, mkdir
from multiprocessing import cpu_count

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from gensim.test.utils import common_texts,get_tmpfile

from gensim import corpora

import re
import pickle
import time
from math import log10,log2
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score

from tqdm import tqdm

In [7]:
with open('artMorphs_final.dat','rb') as file: # 미리 doc별로 저장한 morphs 리스트를 load한다
    tokens = pickle.load(file)

In [6]:
with open('artAd_nouns_final.dat','rb') as file: # 미리 doc별로 저장한 morphs 리스트를 load한다
    tokens = pickle.load(file)

In [6]:
#하이퍼파라미터 세팅
num_features = 20
min_word_count = 3 # 빈도수 3미만 단어 제외
num_workers = cpu_count()
context_size = 10
downsampling = 1e-3
seed=180826

In [8]:
#모델 생성
art2vec =Word2Vec( 
    sg=1, #skip-gram
    workers=cpu_count(),
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample = downsampling,
)

In [9]:
art2vec.build_vocab(tokens)

In [10]:
art2vec.corpus_count

19099

In [11]:
art2vec.epochs

5

In [12]:
%%time
art2vec.train(tokens,total_examples=art2vec.corpus_count,epochs=art2vec.epochs) #학습 with tokens

Wall time: 3min 56s


(49449718, 66756035)

In [59]:
##모델을 저장하고 불러와서 다시 training시킬 수 있다
art2vec.save(r'''art2vec_morphs_100.w2v''')

In [2]:
##전에 학습한 결과 불러오기
art2vec = Word2Vec.load(r'''art2vec_morphs_100.w2v''')

In [4]:
h = art2vec.wv.get_vector('희망')
h

array([-5.86271882e-01, -4.98682082e-01,  2.88966358e-01,  4.34632152e-01,
        4.40805852e-01,  7.20556453e-02, -2.12621853e-01, -3.63508582e-01,
        2.76619017e-01, -1.02190226e-02,  3.61359626e-01, -8.94335628e-01,
       -2.79768169e-01,  1.39855206e-01,  3.95677596e-01,  4.69347030e-01,
       -1.29302725e-01, -2.64756680e-01,  2.72560835e-01,  1.78867906e-01,
        8.35691020e-02,  9.67097059e-02,  3.48771483e-01, -9.12316591e-02,
        2.91012794e-01, -2.41924912e-01,  2.58323491e-01,  8.05526748e-02,
        1.41848996e-01, -3.80138338e-01, -5.96662685e-02,  3.03226560e-01,
       -2.27968648e-01, -1.10423200e-01, -4.99671429e-01, -3.32802981e-01,
        2.16522366e-01,  3.66461463e-02, -1.29108191e-01, -2.80227572e-01,
        2.91184813e-01, -3.15028787e-01, -3.45804542e-01,  3.23616534e-01,
        3.99854302e-01,  6.89490214e-02, -2.50481993e-01,  2.85945386e-01,
       -2.03383669e-01, -3.41408998e-01,  1.03761852e-01,  2.97483783e-02,
       -1.34652123e-01,  

In [6]:
g = art2vec.wv.get_vector('애통')
g

array([-0.10705471, -0.1877016 , -0.11408293,  0.10381389, -0.03186109,
       -0.12837538, -0.01025983, -0.00196287,  0.2567617 , -0.06271862,
        0.0652779 , -0.17916578, -0.05072776,  0.1984345 ,  0.06335732,
        0.3361638 ,  0.00505152, -0.07149127,  0.04875835,  0.26814514,
        0.21542914,  0.09764253,  0.21755937, -0.12861526, -0.08350302,
       -0.06604365,  0.578108  ,  0.05929156,  0.02936027, -0.11066844,
       -0.16861573,  0.24287248,  0.09217749,  0.08736663, -0.02951243,
       -0.32325304,  0.01869313,  0.3440566 , -0.07108182,  0.02914044,
        0.00196177, -0.24809058, -0.13463819, -0.03852449,  0.32108733,
       -0.3407094 , -0.02139591,  0.03293979, -0.05933576, -0.11229908,
       -0.2721232 ,  0.1374884 , -0.00787396,  0.3467139 , -0.06922238,
        0.09075215,  0.05045995, -0.0437403 , -0.04652348,  0.01643862,
       -0.09141186,  0.02470213, -0.01826818, -0.06033063, -0.06291424,
       -0.08034055,  0.07594684,  0.12770854,  0.17315054,  0.04

In [9]:
art2vec.wv.most_similar('부산')

[('서울', 0.7621574401855469),
 ('정인성', 0.7532988786697388),
 ('전남', 0.7336975336074829),
 ('광주', 0.730837345123291),
 ('센텀시티', 0.7307586669921875),
 ('대구', 0.7285851240158081),
 ('인천', 0.7252041101455688),
 ('토요타', 0.7240109443664551),
 ('순천', 0.7217233180999756),
 ('참견', 0.7197357416152954)]

In [17]:
art2vec.wv.most_similar(['가족','반려동물'])

[('가정', 0.8843052387237549),
 ('반려', 0.8769829273223877),
 ('이웃', 0.8763768076896667),
 ('애완동물', 0.8665370941162109),
 ('부모', 0.8658005595207214),
 ('꾸리', 0.8651078343391418),
 ('사랑하는', 0.8510594367980957),
 ('기르는', 0.8375566601753235),
 ('반려견', 0.8372125029563904),
 ('사람', 0.8359472751617432)]

In [18]:
art2vec.wv.most_similar('재미')

[('재미있고', 0.9098564982414246),
 ('재미있는', 0.9006035327911377),
 ('재미있게', 0.8920033574104309),
 ('튀는', 0.8845862150192261),
 ('친근하게', 0.8827311992645264),
 ('흥미', 0.8790271282196045),
 ('다가갈', 0.8672213554382324),
 ('유쾌', 0.8668125867843628),
 ('색다른', 0.8609850406646729),
 ('매력', 0.8590003252029419)]

In [19]:
art2vec.wv.most_similar(['최순민','재미'],negative='김섭')

[('흥미', 0.6404816508293152),
 ('막대사탕', 0.6400307416915894),
 ('재미있는', 0.6391662359237671),
 ('재미있게', 0.6351559162139893),
 ('즐겁', 0.6205161809921265),
 ('디저트', 0.6188223958015442),
 ('재밌는', 0.6089884042739868),
 ('친숙하', 0.6082839965820312),
 ('친근한', 0.6064863801002502),
 ('꺼리', 0.6048434972763062)]

In [20]:
art2vec.wv.most_similar('김혜연','여성')

[('김정', 0.817837119102478),
 ('혁', 0.8131669759750366),
 ('혜경', 0.8026963472366333),
 ('소연', 0.7982786297798157),
 ('비스', 0.7968947887420654),
 ('가나다', 0.7938699126243591),
 ('씨엘', 0.7916637063026428),
 ('정일', 0.7894676923751831),
 ('미현', 0.7891432642936707),
 ('윤', 0.7829801440238953)]

In [21]:
art2vec.wv.most_similar('임만혁','김혜연')

[('아무런', 0.7097944617271423),
 ('별다른', 0.685621440410614),
 ('조차도', 0.6806182265281677),
 ('분명하지', 0.6706468462944031),
 ('필요하지', 0.6683274507522583),
 ('같지', 0.6623393893241882),
 ('않다', 0.6609145402908325),
 ('명확하지', 0.6573029160499573),
 ('확실하지', 0.6572324633598328),
 ('맞지', 0.6548031568527222)]

In [22]:
art2vec.wv.most_similar('','작가')

[('씻어', -0.16750231385231018),
 ('분노', -0.1754438430070877),
 ('압력', -0.17700223624706268),
 ('일삼', -0.17961612343788147),
 ('방사', -0.18627305328845978),
 ('양심', -0.19417491555213928),
 ('방출', -0.19677965342998505),
 ('환자', -0.1971675157546997),
 ('죄책감', -0.1981903314590454),
 ('무능', -0.20407386124134064)]

In [23]:
gm = (art2vec.wv.get_vector('광주') + art2vec.wv.get_vector('미술관') )/2

In [24]:
h = art2vec.wv.get_vector('희망')

In [25]:
art2vec.wv.cosine_similarities(gm,[h]) # 이런식으로 단어 벡터의 평균을 구해 비교할 수 도 있다

array([0.3732139], dtype=float32)

In [4]:
plt.rcParams['font.family'] = 'NanumSquare' #그래프 위한 폰트설정

In [9]:
len(art2vec.wv.vocab)

49947

W2V에 TFIDF로 가중치를 줘보자

In [26]:
with open('TFIDF_morphs_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [27]:
art2vec.trainables.layer1_size #임베딩 벡터사이즈

20

In [28]:
doc_vecs = []
for i, doc in enumerate(tokens):
    mean_vec = np.zeros((art2vec.trainables.layer1_size,)) # 임베딩 벡터의 사이즈크기로 초기화
    doc_len = len(doc)
    for token in doc:
        try:
            mean_vec += art2vec.wv.get_vector(token) * TFIDF[i][token]
        except Exception as e:
#             print(e)
            doc_len -= 1 # w2v 모델에 없는 voca일 경우 제외하며, 이에따라 doc의 word 수도 줄여준다
            continue
    mean_vec = mean_vec/doc_len # doc의 평균 vector를 구한다
    doc_vecs.append(mean_vec)

In [29]:
doc_vecs[0] #word 벡터의 가중평균인 doc 벡터

array([ 0.79390489, -0.37278133,  0.52594747, -0.12335898,  0.27277202,
        0.51923127, -0.82002603, -0.13154508, -0.46785257, -0.49639142,
       -0.29074885, -0.65552008, -1.7143137 ,  0.34396144,  0.65107294,
        0.64271616,  0.07779588, -1.64565271,  0.67982815, -0.46007491])

query doc을 벡터화시켜서 유사한 doc을 찾아보자

In [25]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

In [30]:
query = """“사람들이 나보고 맘충이래.”

한국에서 여자로 살아가는 일
그 공포, 피로, 당황, 놀람, 혼란, 좌절의
연속에 대한 인생 현장 보고서 

조남주 장편소설 『82년생 김지영』이 민음사 ‘오늘의 젊은 작가’ 시리즈로 출간되었다. 조남주 작가는 2011년, 지적 장애가 있는 한 소년의 재능이 발견되면서 벌어지는 사건을 통해 삶의 부조리를 현실적이면서도 따뜻하게 그려낸 작품 『귀를 귀울이면』으로 ‘문학동네소설상’을 받으며 데뷔했다. 시사 교양 프로그램에서 10년 동안 일한 방송 작가답게 서민들의 일상에서 발생하는 비극을 사실적이고 공감대 높은 스토리로 표현하는 데 특출 난 재능을 보이는 작가는 신작 『82년생 김지영』에서 30대를 살고 있는 한국 여성들의 보편적인 일상을 완벽하게 재현한다.
주인공 ‘김지영 씨’의 기억을 바탕으로 한 고백을 한 축으로, 고백을 뒷받침하는 각종 통계 자료와 기사들을 또 다른 축으로 삼는 이 소설은 1982년생 김지영 씨로 대변되는 ‘그녀’들의 인생 마디마디에 존재하는 성차별적 요소를 핍진하게 묘사한다. 이를 통해 작가는 제도적 성차별이 줄어든 시대의 보이지 않는 차별들이 어떻게 여성들의 삶을 제약하고 억압하는지 보여 준다. 
여권이 신장된 시대, 그러나 여전히 ‘여성’이라는 조건이 굴레로 존재하는 사회에서 살아가는 한 여자의 인생을 다룬 『82년생 김지영』은 조용한 고백과 뜨거운 고발로 완성된 새로운 페미니즘 소설이자 수많은 사람들의 경험과 자료로 이루어진 ‘목소리 소설’이다. 맘충이, 여혐, 메갈리아 등 연일 새롭게 등장하는 페미니즘 화두를 관심 있게 지켜보는 독자라면 누구나 흥미롭게 읽을 수 있고 저마다 의미를 발견할 수 있을 것이다. 

■“사람들이 나보고 맘충이래.” 
엄마를 뜻하는 ‘맘(Mom)’과 벌레를 뜻하는 ‘충(蟲)’의 합성어인 ‘맘충’은 제 아이만 싸고도는 일부 몰상식한 엄마를 가리키는 용어다. 그러나 ‘맘충’이란 호칭은 육아하는 엄마 대부분에게 무차별적으로 사용되며 많은 여성들에게 공포심을 주고 상처를 안겼다. 뿐만 아니라 이 표현은 육아가 마치 여성의 일인 것처럼 인식되게 함으로써 성차별적 시선을 고착화하는 데도 일조해 논란의 대상이 되었다. 
『82년생 김지영』은 2014년 말 촉발된 ‘맘충이’ 사건을 목격한 작가가 여성, 특히 육아하는 여성에 대한 사회의 폭력적인 시선에 충격 받아 쓰기 시작한 소설이다. 소설을 쓸 당시 작가는 유치원 다니는 자녀를 둔 전업주부였다. 온라인상에서 사실 관계도 확인되지 않은 상황만 놓고 엄마들을 비하하는 태도에 문제의식을 느낀 작가는 지금 한국을 살아가는 여성들의 삶이 과거에서 얼마나 더 진보했는지, 혹은 그렇지 않은지 질문할 수 있는 이야기를 만들기로 했다. 

■30대 여성들의 인생 보고서
슬하에 딸을 두고 있는 서른네 살 김지영 씨가 어느 날 갑자기 이상 증세를 보인다. 시댁 식구들이 모여 있는 자리에서 친정 엄마로 빙의해 속말을 뱉어 내는 통에 시댁 식구들을 아연실색하게 만드는가 하면 남편의 결혼 전 애인으로 빙의해 그를 식겁하게 만들기도 한다. 이를 이상하게 여긴 남편이 김지영 씨의 정신 상담을 주선하고, 지영 씨는 정기적으로 의사를 찾아가 자신의 삶을 이야기한다.
소설은 김지영 씨의 이야기를 들은 담당 의사가 그녀의 인생을 재구성해 기록한 리포트 형식이다. 리포트에 기록된 김지영 씨의 기억은 ‘여성’이라는 젠더적 기준으로 선별된 에피소드로 구성된다. 발화의 기회가 주어졌을 때 그녀가 선택한 이야기들이 바로 일생에 거쳐 ‘여자이기 때문에 받아 왔던 부당한 일들’이기 때문이다. 
이러한 개인의 고백은 1999년 남녀차별을 금지하는 법안이 제정되고 이후 여성부가 출범함으로써 성평등을 위한 제도적 장치가 마련된 이후, 즉 제도적 차별이 사라진 시대에 보이지 않는 방식으로 존재하는 내면화된 성차별적 요소가 작동하는 방식을 보여 준다. 지나온 삶을 거슬러 올라가며 미처 못다 한 말을 찾는 이 과정은 지영 씨를 알 수 없는 증상으로부터 회복시켜 줄 수 있을까?

■김지영으로 대변되는 젊은 여성들에 대한 섬세한 심리 묘사 
상담은 자기 고백 형식으로 이루어진다. 이 소설의 백미도 김지영 씨의 자기 고백을 중심으로 드러나는 세밀한 심리 묘사다. ‘그때 그 상황’에서는 차마 말하지 못했던 것들을 차분히 쏟아 내는 그녀의 말들은 ‘김지영’을 이 시대 여성의 대변자로 삼기에 충분할 정도로 자세하고 보편적이다. 더욱이 김지영의 이름은 이 시대 젊은 여성들의 삶을 보편적으로 그리기 위한 작가의 전략적 선택이기도 하다. 실제로 1982년에 태어난 여아 중 가장 많이 등록된 이름이 ‘지영’이기 때문이다. 김지영이라는 개인의 고백을 30대 여성, 나아가 이 시대 여성들의 고백으로 볼 수 있는 이유다. 

선배는 평소와 똑같이 다정하고 차분히 물었다. 껌이 무슨 잠을 자겠어요, 라고 대답하고 싶었지만 김지영 씨는 입을 다물어 버렸다. (94쪽)

영업 중인 빈 택시 잡아 돈 내고 타면서 고마워하기라도 하라는 건가. 배려라고 생각하며 아무렇지도 않게 무례를 저지르는 사람. 어디서부터 어디까지 항의를 해야 할지도 가늠이 되지 않았고, 괜한 말싸움을 하기도 싫어 김지영 씨는 그냥 눈을 감아 버렸다. (100~101쪽)

주량을 넘어섰다고, 귀갓길이 위험하다고, 이제 그만 마시겠다고 해도 여기 이렇게 남자가 많은데 뭐가 걱정이냐고 반문했다. 니들이 제일 걱정이거든. 김지영 씨는 대답을 속으로 삼키며 눈치껏 빈 컵과 냉면 그릇에 술을 쏟아 버렸다. (116쪽)

조금도 서운하지 않았다. 견딜 수 없는 것은 오히려 그 순간들이었다. 김지영 씨는 충분히 건강하다고, 약 같은 것은 필요 없다고, 가족 계획은 처음 보는 친척들이 아니라 남편과 둘이 하겠다고 말하고 싶었다. 하지만 아니에요, 괜찮아요, 라는 말밖에 할 수 없었다. (133~134쪽)


■기사, 통계, SNS 등 풍부하고 탄탄한 취재
보고서 형식으로 쓰인 『82년생 김지영』의 에피소드들은 무척이나 사실적이다. 어린 시절, 학창 시절, 회사 생활, 결혼 생활에 이르기까지 여성이라면 누구에게나 익숙한 이 경험들은 온라인 커뮤니티와 SNS에서 많은 공감대를 형성한 사례들을 채집한 결과이기 때문이다. 이러한 구체적인 사례들과 함께 등장하는 각종 팩트들은 지난 20여 년 동안의 ‘성차별 역사’를 한눈에 보여 준다. 『확률 가족』 『기록되지 않은 노동』 『고용 동향 브리프』 등의 도서와 「여자라고 전교 회장 못 하나요」 등의 신문 기사를 비롯해 「인구 동태 건수 및 동태율」 「출산 순위별 출생 성비」 같은 통계청 자료, OECD에서 발표한 성별 인금 격차 (Gender wage gap) 자료 및 외신 기사, 「호주제 페지: 호주제, 벽을 넘어 평등 세상으로」 등 행정부 정책 보고서, 「경력단절 여성 지원정책의 현황과 과제」 같은 보건복지포럼 등의 자료가 쉴 새 없이 등장한다. 개인적 기억과 고백의 형식을 취하고 있는 이야기는 이러한 사실적 자료들을 통해 한국에서 살아가는 여성의 보편적인 삶으로 도약하는 근거가 된다. 


추천글 하나도 낯설지가 않은데 새삼 눈물이 고이다니 이상한 일이다. 눈 돌릴 수 없는 통계와 보도 사이, 그리고 눈에 보이지도 않을 만큼 미세한 차별과 폭력 속에 성장한 나와 내 또래 수많은 지영이들의 삶에 대한 담담하고 서글픈 보고서. 어차피 해피엔딩은 오지 않을 것이다. 다만 운 좋게, 혹은 우연히 살아남은 '여아'들이었던 우리는 이렇게 말하고 기록을 남길 수밖에. -최지은(웹진 《아이즈》 기자)"""

가중치로 쓰기위해 query doc의 TF도 구하자. 문서개수가 하나 뿐인 query doc의 경우 IDF를 구할수 없으므로 빈도수가 곧 가중치가 된다

이는 바람직하지 않으므로 전처리에 신경쓰자

필요한 형태소만 선별하여 가져오고, 2음절 이상 단어만 가져온다. 그리고 불용어 처리까지 해주자

In [31]:
tw = Twitter()
query_tokens = [tag[0] for tag in tw.pos(query) if tag[1] in ['Noun','Adjective','Adverb','Verb'] and len(tag[0]) > 1]
stopwords = ['그','것','이']

In [32]:
query_dic = {}
maxCount = 0

for w in query_tokens:
    if w in stopwords:
        continue
    if w in query_dic.keys():
        query_dic[w] += 1
    else:
        query_dic[w] = 1

    if maxCount < query_dic[w]:
        maxCount = query_dic[w]

print(query_dic)

{'사람': 4, '보고': 2, '맘충': 6, '한국': 4, '여자': 4, '살아가는': 4, '공포': 2, '피로': 1, '당황': 1, '놀람': 1, '혼란': 1, '좌절': 1, '연속': 1, '대한': 4, '인생': 5, '현장': 1, '보고서': 5, '장편소설': 1, '년생': 6, '김지영': 20, '민음사': 1, '오늘': 1, '젊은': 3, '작가': 8, '시리즈': 1, '되었': 2, '지적': 1, '장애': 1, '있는': 7, '소년': 1, '재능': 2, '발견되': 1, '벌어지는': 1, '사건': 2, '통해': 3, '부조리': 1, '현실': 1, '따뜻하게': 1, '그려': 1, '작품': 1, '문학동네': 1, '소설': 8, '받으': 1, '데뷔': 1, '시사': 1, '교양': 1, '프로그램': 1, '동안': 2, '일한': 1, '방송': 1, '서민': 1, '일상': 2, '발생하는': 1, '비극': 1, '사실': 4, '공감': 2, '높은': 1, '스토리': 1, '표현하는': 1, '특출': 1, '보이는': 1, '신작': 1, '에서': 3, '여성': 18, '보편': 4, '완벽하게': 1, '재현': 1, '주인공': 1, '기억': 3, '바탕': 1, '고백': 9, '뒷받침': 1, '하는': 8, '각종': 2, '통계': 3, '자료': 6, '기사': 4, '다른': 1, '삼는': 1, '대변': 2, '되는': 2, '그녀': 4, '마디마디': 1, '존재하는': 3, '성차별': 5, '요소': 2, '진하게': 1, '묘사한': 1, '이를': 2, '제도': 3, '줄어든': 1, '시대': 6, '보이지': 3, '않는': 2, '차별': 4, '어떻': 1, '제약': 1, '억압': 1, '보여': 3, '여권': 1, '신장': 1, '여전히': 1, '조건': 1, '굴레': 1, '사회': 2, '다룬': 1, '조용한'

In [33]:
query_weight = {}
K = 0.5
for k,v in query_dic.items():
    query_weight[k] = K + (1-K) * (v/maxCount)
    print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
            k, K, (1-K), v, maxCount, query_weight[k]
        ))

사람 | 0.5 + 0.5 *(4/20)  =  0.6
보고 | 0.5 + 0.5 *(2/20)  =  0.55
맘충 | 0.5 + 0.5 *(6/20)  =  0.65
한국 | 0.5 + 0.5 *(4/20)  =  0.6
여자 | 0.5 + 0.5 *(4/20)  =  0.6
살아가는 | 0.5 + 0.5 *(4/20)  =  0.6
공포 | 0.5 + 0.5 *(2/20)  =  0.55
피로 | 0.5 + 0.5 *(1/20)  =  0.525
당황 | 0.5 + 0.5 *(1/20)  =  0.525
놀람 | 0.5 + 0.5 *(1/20)  =  0.525
혼란 | 0.5 + 0.5 *(1/20)  =  0.525
좌절 | 0.5 + 0.5 *(1/20)  =  0.525
연속 | 0.5 + 0.5 *(1/20)  =  0.525
대한 | 0.5 + 0.5 *(4/20)  =  0.6
인생 | 0.5 + 0.5 *(5/20)  =  0.625
현장 | 0.5 + 0.5 *(1/20)  =  0.525
보고서 | 0.5 + 0.5 *(5/20)  =  0.625
장편소설 | 0.5 + 0.5 *(1/20)  =  0.525
년생 | 0.5 + 0.5 *(6/20)  =  0.65
김지영 | 0.5 + 0.5 *(20/20)  =  1.0
민음사 | 0.5 + 0.5 *(1/20)  =  0.525
오늘 | 0.5 + 0.5 *(1/20)  =  0.525
젊은 | 0.5 + 0.5 *(3/20)  =  0.575
작가 | 0.5 + 0.5 *(8/20)  =  0.7
시리즈 | 0.5 + 0.5 *(1/20)  =  0.525
되었 | 0.5 + 0.5 *(2/20)  =  0.55
지적 | 0.5 + 0.5 *(1/20)  =  0.525
장애 | 0.5 + 0.5 *(1/20)  =  0.525
있는 | 0.5 + 0.5 *(7/20)  =  0.675
소년 | 0.5 + 0.5 *(1/20)  =  0.525
재능 | 0.5 + 0.5 *(2/2

In [34]:
query_vec = np.zeros((art2vec.trainables.layer1_size,)) # 임베딩 벡터의 사이즈크기로 초기화
query_len = len(query_tokens)

for token in query_tokens:
    try:
        query_vec += art2vec.wv.get_vector(token) * query_weight[token]
    except Exception as e:
            print(e)
            query_len -= 1 # w2v 모델에 없는 voca일 경우 제외하며, 이에따라 query의 word 수도 줄여준다
            continue

query_vec = query_vec/query_len # query의 평균 vector를 구한다
query_vec

"word '맘충' not in vocabulary"
"word '맘충' not in vocabulary"
"word '여혐' not in vocabulary"
"word '갈리아' not in vocabulary"
"word '맘충' not in vocabulary"
"word '맘충' not in vocabulary"
"word '몰상식' not in vocabulary"
"word '맘충' not in vocabulary"
"word '일조해' not in vocabulary"
"word '맘충' not in vocabulary"
"word '실색' not in vocabulary"
"word '식겁' not in vocabulary"
"word '여성부' not in vocabulary"
"word '자겠' not in vocabulary"
"word '주량' not in vocabulary"
"word '갓길' not in vocabulary"
"word '브리프' not in vocabulary"
"word '전교' not in vocabulary"
"word '건수' not in vocabulary"
"word '인금' not in vocabulary"
"word '호주제' not in vocabulary"
"word '페지' not in vocabulary"
"word '호주제' not in vocabulary"


array([ 0.03027064, -0.04531055,  0.17449326, -0.09768855,  0.13478803,
        0.10777373, -0.20669363, -0.30981364, -0.13871299,  0.01996128,
        0.24963995, -0.04303289, -0.23847198,  0.11062611,  0.18124251,
        0.20702789,  0.1845644 , -0.32270996,  0.07317651, -0.0584573 ])

In [35]:
sims = []
for doc_vec in doc_vecs:
    sim = np.dot(query_vec, doc_vec)/(np.linalg.norm(query_vec)*np.linalg.norm(doc_vec))
    sims.append(sim)

In [36]:
sims = np.array(sims)

In [37]:
sims_result = list(zip(np.argsort(sims)[::-1], np.sort(sims)[::-1]))

In [64]:
d19099 = pd.read_excel('d19099.xlsx',sheet_name='Sheet1')

In [94]:
for idx, sim in sims_result[:10]:
    print(d20514.iloc[idx,:].전시제목,d20514.iloc[idx,:].전시작가,' 유사도: ',sim,'\n',d20514.iloc[idx,:]['링크'])

The epistle of the Others - 소정수展 :: Photography ['소정수(Soh Jungsoo)']  유사도:  0.9839799292579733 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=7499
자연:기억 속의 풍경 - 이종갑展 :: Painting ['이종갑(Lee Jonggab)']  유사도:  0.9824272258479398 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=16583
BLUE - 김태균展 :: Photography ['김태균(Kim Taekyun)']  유사도:  0.9821150302499443 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=12976
소라II 삶-생명의 노래 - 엄길자展 :: Painting ['엄길자(Um Kilja)']  유사도:  0.9815979659563902 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=1224
최병관展 :: Photography ['최병관(Choi Byungkwan)']  유사도:  0.9813819848896677 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=1318
THE LIGHT - 오치균展 :: Painting ['오치균(Oh Chigyun)']  유사도:  0.9813535605261826 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=16923
김준권展 :: Mokpan ['김준권(Kim Joonkwon)']  유사도:  0.9809557413226639 
 http://www.arthub.co.kr/sub01/board03_view.htm?No=13564
항금리 가는 길 - 김인옥展 :: Painting ['김인옥(Kim Inok 金仁玉)']  유사도:  0.980876

### exhbDoc2Vec

In [166]:
with open('artMorphs_final.dat','rb') as file:
    tokens = pickle.load(file)

In [172]:
documents = [TaggedDocument(doc, [i]) for i,doc in enumerate(tokens)]

In [173]:
documents[-1]

TaggedDocument(words=['무엇', '인가', '요동', '치고', '술렁거리', '고', ',', '무언가', '는', '급한', '속도', '로', '내', '달리', '고', ',', '또', '다른', '무언가', '는', '들썩', '거리', '며', '꿈틀', '댄', '다', '.', '색채', '의', '난장', '이', '화면', '위', '에서', '벌어지', '고', '있다', '.', '이런', '첫', '인상', '은', '숨길', '수가', '없다', '.', '격동', '의', '시간', '이', '지나자', '화면', '이', '곧', '잠잠', '해지', '며', '섬섬', '하게', '그려진', '실체', '들', '이', '망막', '에', '포착', '된', '다', '.', '분명', '거기', '에는', '화사한', '꽃', '이', ',', '다소', '튀는', '색깔', '들', '로', '생', '경하', '게', '재', '해석했', '음에도', '거리', '낌', '없이', '내', '달리', '는', '말', '이', '경쾌', '하게', '자리', '한', '다', '.', '까닭', '에', '사건', '의', '전후', '를', '고려하자', '면', '그', '가', '그려', '낸', '아니', '체현', '한', '형상', '은', '일순간', '에', '낚아챈듯', '하고', '임의', '적', '묘사', '의', '성격', '도', '강하', '지만', ',', '동시', '에', '역설', '적', '으로', '대상', '본래', '의', '형태', '에', '심히', '빚진', '게', '아닌', '가', '할', '정도', '로', '섬세', '하게', '묘사', '된', '듯', '한', '표현', '력', '도', '엿보', '이', '지만', '이는', '작가', '와', '대상', '간의', '관계', '에', '대한', '성찰', '을', '통해', '물질', '을',

In [224]:
#하이퍼파라미터 세팅
max_epochs = 20

vec_size = 100
window_size = 8
alpha = 0.025
min_alpha=0.00025
min_count=3
sampling_threshold = 1e-5
negative_size = 5

In [225]:
model = Doc2Vec(
                vector_size=vec_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                sample = sampling_threshold,
                negative = negative_size,
                dm =1 #If dm=1, ‘distributed memory’ (PV-DM) is used .Otherwise, distributed bag of words (PV-DBOW) is employed.
                            # In this case, PV-DM is the better 
               )

In [226]:
model.build_vocab(documents)

In [227]:
for epoch in tqdm(range(max_epochs)):
#     print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_exhb_50.model")
print("Model Saved")



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

 10%|████████▎                                                                          | 1/10 [01:10<10:30, 70.03s/it]

 20%|████████████████▌                                                                  | 2/10 [02:33<10:12, 76.62s/it]

 30%|████████████████████████▉                                                          | 3/10 [03:54<09:07, 78.19s/it]

 40%|█████████████████████████████████▏                                                 | 4/10 [05:11<07:47, 77.89s/it]

 50%|█████████████████████████████████████████▌                                         | 5/10 [06:26<06:26, 77.23s/it]

 60%|█████████████████████████████████████████████████▊                                 | 6/10 [07:54<05:16, 79.16s/it]

 70%|██████████████████████████████████████████████████████████                         | 7/10 [09:17<03:59, 79.71s/it]

 80%|█████████████████████████

Model Saved


In [2]:
%%time
model = Doc2Vec.load("d2v_exhb_100.model")

Wall time: 1.35 s


In [241]:
model.wv.most_similar('여성')

[('남성', 0.5910496711730957),
 ('여성성', 0.540796160697937),
 ('기녀', 0.5366159081459045),
 ('성기', 0.4948946237564087),
 ('나체', 0.4815532863140106),
 ('성적', 0.46681153774261475),
 ('히스패닉', 0.4598768353462219),
 ('남성혐오', 0.4557049870491028),
 ('여인', 0.45075830817222595),
 ('남성성', 0.4491884112358093)]

In [242]:
model.wv.most_similar('고흐')

[('빈센트', 0.6218597292900085),
 ('말하길', 0.5358984470367432),
 ('고호', 0.5235546827316284),
 ('마티스', 0.4974503219127655),
 ('1853', 0.4765508770942688),
 ('게르니카', 0.46304911375045776),
 ('아비뇽', 0.44178488850593567),
 ('1832', 0.43950381875038147),
 ('보이스', 0.43844130635261536),
 ('베토벤', 0.437512069940567)]

In [243]:
model.wv.most_similar('대구')

[('부산', 0.6419674158096313),
 ('북성로', 0.5258275866508484),
 ('칠곡', 0.5167964696884155),
 ('달서구', 0.5053877234458923),
 ('회관', 0.49761444330215454),
 ('광주', 0.4954257607460022),
 ('영남', 0.48812830448150635),
 ('충청', 0.47785240411758423),
 ('배밭', 0.47577884793281555),
 ('요코하마', 0.46870657801628113)]

In [244]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

In [251]:
query = """미술관은 2013년 창작스튜디오 입주작가 8명의 그룹전시인 8 을 오는 1월 23일부터 2월 23일까지 32일 동안 개최한다. 이번 전시는 지난 2011년 4월부터 운영 중인 인천광역시 학익동 소재 창작스튜디오에 3기 작가로 입주하여 작품 활동에 주력해온 8명 작가의 레포팅 전시이다. 3기 선정 작가 8명은 권오신, 김유정복합매체, 김희연, 박미경, 박종호서양화, 이주은복합매체, 조문희영상, 허용성한국화으로, 2013년 2월에 공개모집으로 선정되었다. 2013년 10월 26일토부터 28일월까지 3일간 오픈스튜디오 개최 를 통해 미술계 인사들 및 방문객과의 소통을 시도한 바 있다. 8 전은 8인의 입주작가들이 그동안 쌓아온 열정의 결실을 한 자리에서 살펴보기 위해 마련되었다. 8 는 입주작가 보고전의 새로운 타이틀이다. 이는 미술관 창작스튜디오가 가장 중요한 목표로 추구하고 있는 작가 8인의 창의적 작품 활동과, 긍정적 네트워크를 위한 공간이라는 의미를 내포한다. 권오신은 어린 시절 행복했던 기억에서 작품을 시작한다. 석판화위에 레진을 붓고 판화를 오려낸 콜라주를 부착하는 과정을 반복한다. 할머니의 기와집, 원숭이, 풍선 등의 소재들은 과거부터 현재까지 여러 층의 시간에서 얻어진 소재들로, 기억들은 생성, 소멸이 중첩되어 새로운 시?공간을 형성한다. 이를 통해 우리의 기억으로 공유될 수 있는 생명력을 가진다. 김유정은 관상식물의 수동적인 생육 방식을 인간의 삶에 비춰 사유한다. 전통 프레스코 기법을 차용하여 회벽이 마르기 전에 긁어내어 형상을 표현한다. 단색조의 화분들은 무기력하고 상처받은 자아를 상기시킨다. 또한 야생처럼 보이는 것들도 실제로는 식물원에 갇혀있는 아이러니한 상황임을 나타낸다. 그러나 끊임없이 긁어내며 생명을 불어넣는 행위를 통해 자아의 주체성 회복과 치유를 모색한다.김희연은 도시의 풍경에서 기이한 느낌을 자아내는 공간들에 주목한다. 주로 사라져가는 건물, 공사장 등에 자연물이 공존하는 풍경을 회화로 표현한다. 사람의 부재, 빛과 그림자의 대비와 차분한 붓질은 적막한 초현실적 풍경을 만들어낸다. 이를 통해 인간의 행위와 시간, 자연이 만들어내는 미묘한 지점에 맞닿아 있는 내면을 표현한다.박미경은 기억의 심연이 지닌 다층성을 회화로 표현한다. 주로 어두운 색조를 무수한 붓질로 중첩시킨다. 폐허의 공간 또는 무언가 살아나는 것 같은 풍경은 작가의 아픈 기억에 기반 한 것이지만, 현재의 여러 경험들에 의해 변화됨을 보인다. 박미경의 작품에서 기억은 생성에서 소멸로 또 생성으로 끝없이 이어지며 자라난다. 박종호는 회화를 통해 무엇을 왜 그리는가에 대해 깊이 연구해왔는데, 현재는 그 시각을 보다 확장하였다. 창밖의 하늘 풍경에서 시작하여 타인과 외부의 요소들에 대한 자유로운 회화적 열망을 담아낸다. 최근에는 입체, 영상에 이르는 새로운 매체를 시도하고 있으며, 정치적 이슈, 언어와 회화의 관계 등 다양한 영역을 성찰하고 있다.이주은은 익숙한 사물에 대한 새로운 시각을 탐구한다. 작가는 사물의 부분을 포착하여 연출 사진을 찍고 레진을 부은 후 드로잉하는 과정을 반복한다. 의자, 침대 다리 등의 사물은 전혀 다른 형태로 구성되는데, 사물에 담긴 사람의 흔적과 시간이 암시된다. 이주은의 작품은 일상의 소박한 단면들에 잠재된 경이로움을 일깨워준다.조문희는 현실세계를 둘러싼 요소들의 실재와 허구에 관한 문제를 사진과 영상을 통해 탐구한다. 주로 주변의 건물, 도시 풍경을 사진 촬영한 후 이정표 등을 모두 삭제하거나 흐리게 하여 정체성이 불분명한 공간을 형성한다. 이를 통해 오히려 지워진 존재를 상기시키며, 피상적인 공간에서 살아가는 현대인의 소외감을 암시하고자 한다. 허용성은 젊은 세대가 겪는 방황에 관한 주제를 인물화로 연구해왔다. 한지에 채색을 덧입히는 섬세한 과정을 통해 주로 한 작품에 한 인물만을 표현한다. 핏기 없는 피부, 불안한 눈빛을 지닌 인물들로 길을 잃은 세대들에 대한 공감을 표현한다. 최근에는 반신과 전신으로 확대되고, 배경과 조금씩 호흡하는 작품들로 변화되고 있다. 을 통해 관람객들은 8인 8색의 작품세계를 깊이 이해할 수 있는 기회가 될 것이다. 미술관 창작스튜디오는 앞으로도 지속적인 관심을 가지고 작가들과 함께 호흡하는 시각 예술 공간으로 자리매김하고자 한다. """

In [252]:
query_tokens = tw.morphs(query)

In [253]:
new_vector = model.infer_vector(query_tokens,steps=5) 
sims = model.docvecs.most_similar([new_vector])
sims

[(1, 0.856023907661438),
 (2614, 0.5138183832168579),
 (14281, 0.49000659584999084),
 (4341, 0.4799574315547943),
 (15738, 0.47093069553375244),
 (15538, 0.46471765637397766),
 (8326, 0.45677605271339417),
 (99, 0.4562988877296448),
 (7368, 0.4559359848499298),
 (12593, 0.4536331295967102)]

In [254]:
' '.join(tokens[2614])

'두산 갤러리 서울 에서는 2014 년 11 월 19 일 부터 12 월 31 일 까지 2012 년 제 3 회 두산 연강 예술상 수상자 인 김상돈 , 김지은 , 장 지아 의 두산 연강 예술상 제 3 회 수상작 가 그룹 전 을 개최 한 다 . 세 작가 는 2014 년 상반기 두산 레 지 던 시 뉴욕 에 입 주 하 여 개인 전과 레 지 던 시 프로그램 에 참여하였 다 . 신작 위주 로 선 보이 게 될 이번 전시 는 레 지 던 시 입 주 기간 동안 작품 의 변화 과정 을 살펴볼 수 있 을 것 이다 . 김상돈 은 뉴욕 의 레 지 던 시 입 주 기간 동안 파편 화 된 사물 인간 도구 들 이 서로 연결 신호 를 송수 신 하 기 위해 의지하며 서 있는 군상 들 의 모습 을 오브 제로 만든 안테나 연작 을 선 보였 다 . 이후 한국 에서 발생한 세월호 , 싱크홀 등 일련 의 사건 들 을 목격 하며 , 모뉴먼트 제로 를 통해 부재 를 역설 적 으로 상징하는 기념비 를 설치하고 이를 사진 으로 담기 도 했 다 . 신작 입 성운 시리즈 에서는 안테나 , 모뉴먼트 제로로 부터 이어진 작가 의 시공간 의 유연성 에 대한 관심 과 탐구 과정 을 볼 수 있 을 것 이다 . 김지은 은 도시 개발 과 고속 성장 으로 빠르 게 신축 되 고 폐기 되는 건축 현장 의 모습 을 페인팅 과 설치 로 담는 다 . 이번 전시 에서 선 보이는 설치 작품 만능 컨테이너 프로젝트 시리즈 는 가 건축물 , 화물 수송 , 바리 케이트 등 다용 도로 사용되는 컨테이너 의 다양한 모습 을 통해 개발 과 폐기 의 순환 이 끊임없 이 진행되는 도시화 의 단면 과 이중성 을 일 깨워 준 다 . 장지 아는 사회 의 금기 를 인간 의 신체 를 매체 로 퍼포먼스 , 사진 , 영상 , 설치 등 으로 표현한 다 . 고통 과 쾌락 , 폭력 성과 아름다 움 등 의 상반 된 감각 이 공존 하는 작품 을 통해 인간 내부 에 잠재 된 본능 을 되돌아보도 록 만든 다 . 이번 전시 에서는 동물 의 가죽 에 인두 로 이미지 나 문장 을 새긴 

### artistsDoc to Vec

In [255]:
with open('artists_twice_morphs_final.lst','rb') as file:
    artists_twice_tokens = pickle.load(file)

In [111]:
# with open('artists_twice_adnouns_final.lst','rb') as file:
#     artists_twice_tokens = pickle.load(file)

In [288]:
documents = [TaggedDocument(artist[1], [artist[0]]) for artist in artists_twice_tokens]

In [146]:
documents[-1]

TaggedDocument(words=['이번', '전시', '는', '2012', '년', '에', '시작된', '1', '이', '필두', '가', '되었', '으며', ',', '작가', '로서', '새로운', '방향성', '을', '모색하고', '그', '간의', '작업', '을', '발표하기', '위한', '자리', '로써', '마련되', '었', '습니다', '.', '2', '은', '마주', '하고', '있는', '예술', '환경', '에', '대한', '보편', '적', '접근', '방식', '에', '머무르', '지', '않고', ',', '창작', '의', '주체', '인', '작가', '들', '이', '모여', '각자', '의', '프로세스', '를', '공유', '하고', '토론하는', '향유', '의', '장이', '되었', '습니다', '.', '그동안', '의', '에너지', '가', '빚어', '낸', '2', '은', '현대', '의', '예술', '문화', '에', '대한', '질서', '와', '담론', '을', '논의하고', ',', '새로운', '시대', '에', '실현될', '예술', '을', '조명하', '는', '플랫폼', '이라', '할', '수', '있', '습니다', '.', '15', '명의', '작가', '들', '이', '선', '보이는', '다양한', '작업', '을', '통하여', '동시', '대', '신진', '작가', '들', '의', '감성', '과', '시선', ',', '그리고', '그', '작품', '들', '이', '지니', '고', '있는', '진정', '한', '의미', '를', '전하고', '소통', '하는', '계기', '가', '되', '기를', '기원', '합', '니다', '.', '권용무', '현대', '사회', '에서', '생산되', '어', '진', '기계', '부속', '물의', '해체', '와', '조합', ',', '확장', '을', '통해', '새로운', '시각'

In [289]:
#하이퍼파라미터 세팅
max_epochs = 20

vec_size = 100
window_size = 15
alpha = 0.025
min_alpha=0.00025
min_count=3
sampling_threshold = 1e-5
negative_size = 5

In [290]:
model = Doc2Vec(
                vector_size=vec_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                dm =1 #If dm=1, ‘distributed memory’ (PV-DM) is used .Otherwise, distributed bag of words (PV-DBOW) is employed.
               )

In [291]:
model.build_vocab(documents)

In [293]:
for epoch in tqdm(range(max_epochs)):
#     print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_artist_100.model")
print("Model Saved")



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

  5%|████                                                                              | 1/20 [01:41<32:10, 101.60s/it]

 10%|████████▏                                                                         | 2/20 [03:24<30:36, 102.04s/it]

 15%|████████████▎                                                                     | 3/20 [05:02<28:31, 100.68s/it]

 20%|████████████████▌                                                                  | 4/20 [06:38<26:35, 99.73s/it]

 25%|████████████████████▌                                                             | 5/20 [08:21<25:04, 100.30s/it]

 30%|████████████████████████▌                                                         | 6/20 [10:07<23:38, 101.31s/it]

 35%|████████████████████████████▋                                                     | 7/20 [11:50<21:58, 101.43s/it]

 40%|█████████████████████████

Model Saved


In [294]:
model = Doc2Vec.load("d2v_artist_100.model")

In [130]:
len(model.docvecs)

8892

In [295]:
model.wv.most_similar('대구')

[('우리나라', 0.49754321575164795),
 ('광주', 0.4927663803100586),
 ('청주', 0.4800831377506256),
 ('한국', 0.45885980129241943),
 ('배다리', 0.44781482219696045),
 ('플랑드르', 0.44620758295059204),
 ('영남', 0.4352894127368927),
 ('일민', 0.4039663076400757),
 ('대한민국', 0.3957783579826355),
 ('수도권', 0.39267346262931824)]

In [296]:
model.wv.most_similar('광주')

[('대구', 0.49276643991470337),
 ('청주', 0.4680896997451782),
 ('인천', 0.421459436416626),
 ('이사장', 0.4033546447753906),
 ('울산', 0.40182769298553467),
 ('배다리', 0.3976871371269226),
 ('백석', 0.39562004804611206),
 ('품다', 0.393731027841568),
 ('대전', 0.3900158107280731),
 ('공주시', 0.3870391547679901)]

In [297]:
model.wv.most_similar('고흐')

[('행로', 0.45029908418655396),
 ('빈센트', 0.4448249340057373),
 ('데카르트', 0.42218226194381714),
 ('순애보', 0.4165082573890686),
 ('보시', 0.41242295503616333),
 ('바르', 0.4110562205314636),
 ('712770', 0.4081326127052307),
 ('권영성', 0.3972717821598053),
 ('바이칼호수', 0.3878892660140991),
 ('탯줄', 0.38692396879196167)]

In [298]:
model.wv.most_similar('남성')

[('여고생', 0.44669419527053833),
 ('여성', 0.4396127462387085),
 ('재령', 0.4278995990753174),
 ('몸매', 0.41408511996269226),
 ('발톱', 0.40999090671539307),
 ('무신론', 0.40984347462654114),
 ('백인', 0.40371453762054443),
 ('콤플렉스', 0.4036245048046112),
 ('칡', 0.3999253511428833),
 ('가난', 0.39975690841674805)]

In [299]:
model.wv.most_similar('여성')

[('여인', 0.5794418454170227),
 ('사람', 0.5274228453636169),
 ('젊은이', 0.49525293707847595),
 ('현대인', 0.4806057810783386),
 ('인간', 0.4794159531593323),
 ('소녀', 0.4676758050918579),
 ('영웅', 0.4651709198951721),
 ('한국인', 0.45845091342926025),
 ('남성', 0.4396127462387085),
 ('선수', 0.4173760414123535)]

In [300]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

In [301]:
tw = Twitter()
query_tokens = tw.morphs(query)

In [303]:
new_vector = model.infer_vector(query_tokens,steps=100) #
sims = model.docvecs.most_similar([new_vector])
sims

[('윤상렬', 0.6417140960693359),
 ('김윤수', 0.6328012943267822),
 ('윤정원', 0.6277910470962524),
 ('지석철', 0.6210947036743164),
 ('박현주(Park Hyunjoo)', 0.6198165416717529),
 ('김범석', 0.6196178197860718),
 ('홍명섭', 0.6185243129730225),
 ('전현선', 0.6184067726135254),
 ('유근택', 0.6111027598381042),
 ('임희성', 0.6102385520935059)]

In [304]:
[' '.join(artist[1]) for artist in artists_twice_tokens if artist[0]=='윤정원']

['동시 대인 이란 자신 의 시대 에 시선 을 고정 함 으로써 빛 이 아니 라 어둠 을 지각 하는 자 이다 . 모든 시대 는 그 동시 대성 을 체험하는 자 들 에게는 어둡 다 . 따라서 동시 대인 이란 이 어둠 을 볼 줄 아는 자 , 펜 을 현재 의 암흑 에 담그 며 써 내려갈 수 있는 자 이다 . 조르 주 아 감 벤 컨템포러리 의 물결 위 에 지금 우리 시대 는 물질 인지 자본 의 홍수 속 에서도 언제나 갈증 을 호소 한 다 . 전 에 없던 새 롭고 신선한 것 , 흥미로 우 면서도 진지한 것 , 단 번 에 시선 을 끌 수 있는 자극 적 이고 충격 적 인 것 들 이 우위 를 점 하면 서 가치 는 매일 전복 되 고 있다 . 창조 는 혼성 모방 이나 차용 과 혼동 되 며 유행 을 낳고 , 우리 가 먹고 마시 고 입고 노 는 관습 의 세계 는 이벤트 를 갈망 한 다 . 새로운 아이디어 가 경쟁 이 되는 이 세계 에서 오늘 의 새 로움 은 곧 내일 의 진부 함 으로 금새 퇴색 된 다 . 2000 년대 이후 글로벌 경쟁 사회 에 돌입 하면 서 문화 예술 의 생산 , 교류 도 전 지구 적 차원 으로 진행되 어 오 고 있으 며 그 에 대한 반응 의 속도 와 폭 은 더 즉각 적 이고 넓어졌 다 . 그럼 으로써 새로운 비평 관점 , 새로운 예술 양식 에 대한 갈망 은 20 세기 이래 최고 조 에 달 하 는 것 같다 . 20 세기말 포스트모더니즘 논쟁 이 느슨 해 지기 무섭 게 21 세기 첫 10 년 이 지나면 서 시각예술 은 또 다른 이즘 으로 동시 대 를 규정 하기 도 한 다 . 하나 의 이즘 이 비판이론 에 부딪히 면서 제 3 의 이즘 으로 대안 을 형성 해왔 던 변증법 적 전개 방식 은 , 이제 유사한 담론 이 동시 다발 적 으로 생 성 반복되는 동시 대 미술 흐름 의 곡선 에 적용하기 어렵 게 되었 다 . 20 세기 말 부터 회 자 되었 던 컨템포러리 아트 의 성격 규명 은 현재 도 진행중 이다 . 21 세기 인들 은 세기 의 첫 10 년 을 포스트 포스트 모더니즘 ,

In [156]:
[' '.join(artist[1]) for artist in artists_twice_tokens if artist[0]=='김영진']

['처음 이 작품 을 만난 날 그 앞 에서 난 눈 을 땔 수가 없었 다 . 축축 하고 음산 한 영국 의 날씨 를 그대로 옮겨 놓 은 듯 그로테스크 한 유럽 의 전설 에나 나올 법 한 낡은 교회 는 안개 인지 낮게 낀 구름 인지 에 끝 이 보이지 않고 담쟁이 넝굴 사이 로 보이는 어둠 은 너무나 깊어 서 원 히 모든 빛 을 빨 아 들일 것 만 같다 . 어둡 고 축축 한 그리 유쾌하 지 않은 감정 들 이 이렇게 고상 하고 품위 있게 묘사 될 수 있구 나 . 이 작품 을 볼 때 마다 가슴 깊숙이 감춰 둔 비밀스런 감정 들 이 스멀스멀 올라오는 게 느껴진 다 . 싸 하 지만 불쾌 하지 않고 울렁거리 지만 메스껍 지 않은 강한 카타르 시스 가 . 한남동 의 카페 초능력 에서 있었 던 에테르 작가 의 개인 전 준비 를 위한 호소 의 전시 초능력 에서 우정 의 무대 사심 전 . 이 독특하고 재능 넘치는 작가 는 나무 문짝 한쪽 에 프랑스 악동 , 다른 한쪽 에 시조새 가 경쾌 하게 그려진 작품 을 내 앞 에 들이 밀 며 말했 다 . 이건 꼭 사셔 야 해 요 . 세계 최초 양면 그림 이 거든요 세계 최초 라는 멘트 는 그 작품 특유 의 작품 성과 스토리텔링 을 초월 해 관객 으로 하여금 주머니 를 열게 만들었 다 . 세계 최초 양면 그림 . 음 쿨 한 데 ? 작가 니 임 저 랑 미키 를 그려 주 세요 난 한 작품 을 주문했 을 뿐 인데 지금 까지 본 가장 행복한 표정 의 멍멍 이 라며 작가 님 은 손바닥 그림 시리즈 여기저기 에 미키 녀석 을 그려 넣었 다 . 그 그림 들 이 며 다른 전시 에서도 인기 가 많은 걸 보니 왠지 연예인 엄마 와 같은 기분 이 드는 걸 ? 거리 의 풍경 을 어찌나 잘 그리는 지 멋진 거리 에서 맘껏 그려 보라 고 이태리 피렌체 에 사는 친구 가 여원 작가 를 초대했 다 . 작가 는 하루 종일 여기 저 길 돌아다니 다 강한 영감 과 그려 야 겠 다는 욕망 이 일어나면 거기 가 어디가 됐 든 그 자리 에 주저앉 아 그림 을 그렸 을 것 이다 . 봐 

In [157]:
query = """뉴욕 타임스」, 「선데이 타임스」와 아마존의 베스트셀러 리스트를 석권한 영국 작가 앤서니 호로비츠의 장편소설. 영미권에서만 30만 부 이상의 판매고를 올렸다. 미스터리와 서스펜스의 대가로 소설, 시나리오, 드라마, 아동.청소년 문학 등 장르를 넘나들며 다양한 독자층을 확보한 호로비츠의 작품들은 전 세계 30개 이상의 언어로 번역되고 BBC와 ITV 텔레비전 시리즈로 제작되어 큰 인기를 끌었다. 

2016년 출간된 <맥파이 살인 사건>은 고전 탐정 소설의 황금시대를 재현한 추리 소설이다. 같은 해 「뉴욕 타임스」, 「선데이 타임스」 및 아마존 베스트셀러로 독자들의 열광을 불러일으키고 「워싱턴 포스트」, 「에스콰이어」, NPR 선정 '올해 최고의 책'으로 꼽혔으며 「데일리 메일」 '올해 최고의 범죄 소설'에 선정됐다.

별고 없는 조용한 마을 색스비온에이번에서 대저택 파이 홀의 가정부 메리 블래키스턴의 장례식이 치러진다. 추도식을 맡은 목사, 음흉한 앤티크 숍 주인, 고인과 갈등을 겪은 아들, 시신을 발견한 관리인 등 등장인물들의 미심쩍은 행동과 죽음을 둘러싼 소문들이 밀도 있게 다뤄진다. 이후 파이 홀의 주인인 매그너스 파이마저 기이한 죽음을 맞는다. 소식을 접한 탐정 아티쿠스 퓐트는 본격적으로 수사에 착수한다."""

도서 서평을 넣어보자

In [161]:
query = """“사람들이 나보고 맘충이래.”

한국에서 여자로 살아가는 일
그 공포, 피로, 당황, 놀람, 혼란, 좌절의
연속에 대한 인생 현장 보고서 

조남주 장편소설 『82년생 김지영』이 민음사 ‘오늘의 젊은 작가’ 시리즈로 출간되었다. 조남주 작가는 2011년, 지적 장애가 있는 한 소년의 재능이 발견되면서 벌어지는 사건을 통해 삶의 부조리를 현실적이면서도 따뜻하게 그려낸 작품 『귀를 귀울이면』으로 ‘문학동네소설상’을 받으며 데뷔했다. 시사 교양 프로그램에서 10년 동안 일한 방송 작가답게 서민들의 일상에서 발생하는 비극을 사실적이고 공감대 높은 스토리로 표현하는 데 특출 난 재능을 보이는 작가는 신작 『82년생 김지영』에서 30대를 살고 있는 한국 여성들의 보편적인 일상을 완벽하게 재현한다.
주인공 ‘김지영 씨’의 기억을 바탕으로 한 고백을 한 축으로, 고백을 뒷받침하는 각종 통계 자료와 기사들을 또 다른 축으로 삼는 이 소설은 1982년생 김지영 씨로 대변되는 ‘그녀’들의 인생 마디마디에 존재하는 성차별적 요소를 핍진하게 묘사한다. 이를 통해 작가는 제도적 성차별이 줄어든 시대의 보이지 않는 차별들이 어떻게 여성들의 삶을 제약하고 억압하는지 보여 준다. 
여권이 신장된 시대, 그러나 여전히 ‘여성’이라는 조건이 굴레로 존재하는 사회에서 살아가는 한 여자의 인생을 다룬 『82년생 김지영』은 조용한 고백과 뜨거운 고발로 완성된 새로운 페미니즘 소설이자 수많은 사람들의 경험과 자료로 이루어진 ‘목소리 소설’이다. 맘충이, 여혐, 메갈리아 등 연일 새롭게 등장하는 페미니즘 화두를 관심 있게 지켜보는 독자라면 누구나 흥미롭게 읽을 수 있고 저마다 의미를 발견할 수 있을 것이다. 

■“사람들이 나보고 맘충이래.” 
엄마를 뜻하는 ‘맘(Mom)’과 벌레를 뜻하는 ‘충(蟲)’의 합성어인 ‘맘충’은 제 아이만 싸고도는 일부 몰상식한 엄마를 가리키는 용어다. 그러나 ‘맘충’이란 호칭은 육아하는 엄마 대부분에게 무차별적으로 사용되며 많은 여성들에게 공포심을 주고 상처를 안겼다. 뿐만 아니라 이 표현은 육아가 마치 여성의 일인 것처럼 인식되게 함으로써 성차별적 시선을 고착화하는 데도 일조해 논란의 대상이 되었다. 
『82년생 김지영』은 2014년 말 촉발된 ‘맘충이’ 사건을 목격한 작가가 여성, 특히 육아하는 여성에 대한 사회의 폭력적인 시선에 충격 받아 쓰기 시작한 소설이다. 소설을 쓸 당시 작가는 유치원 다니는 자녀를 둔 전업주부였다. 온라인상에서 사실 관계도 확인되지 않은 상황만 놓고 엄마들을 비하하는 태도에 문제의식을 느낀 작가는 지금 한국을 살아가는 여성들의 삶이 과거에서 얼마나 더 진보했는지, 혹은 그렇지 않은지 질문할 수 있는 이야기를 만들기로 했다. 

■30대 여성들의 인생 보고서
슬하에 딸을 두고 있는 서른네 살 김지영 씨가 어느 날 갑자기 이상 증세를 보인다. 시댁 식구들이 모여 있는 자리에서 친정 엄마로 빙의해 속말을 뱉어 내는 통에 시댁 식구들을 아연실색하게 만드는가 하면 남편의 결혼 전 애인으로 빙의해 그를 식겁하게 만들기도 한다. 이를 이상하게 여긴 남편이 김지영 씨의 정신 상담을 주선하고, 지영 씨는 정기적으로 의사를 찾아가 자신의 삶을 이야기한다.
소설은 김지영 씨의 이야기를 들은 담당 의사가 그녀의 인생을 재구성해 기록한 리포트 형식이다. 리포트에 기록된 김지영 씨의 기억은 ‘여성’이라는 젠더적 기준으로 선별된 에피소드로 구성된다. 발화의 기회가 주어졌을 때 그녀가 선택한 이야기들이 바로 일생에 거쳐 ‘여자이기 때문에 받아 왔던 부당한 일들’이기 때문이다. 
이러한 개인의 고백은 1999년 남녀차별을 금지하는 법안이 제정되고 이후 여성부가 출범함으로써 성평등을 위한 제도적 장치가 마련된 이후, 즉 제도적 차별이 사라진 시대에 보이지 않는 방식으로 존재하는 내면화된 성차별적 요소가 작동하는 방식을 보여 준다. 지나온 삶을 거슬러 올라가며 미처 못다 한 말을 찾는 이 과정은 지영 씨를 알 수 없는 증상으로부터 회복시켜 줄 수 있을까?

■김지영으로 대변되는 젊은 여성들에 대한 섬세한 심리 묘사 
상담은 자기 고백 형식으로 이루어진다. 이 소설의 백미도 김지영 씨의 자기 고백을 중심으로 드러나는 세밀한 심리 묘사다. ‘그때 그 상황’에서는 차마 말하지 못했던 것들을 차분히 쏟아 내는 그녀의 말들은 ‘김지영’을 이 시대 여성의 대변자로 삼기에 충분할 정도로 자세하고 보편적이다. 더욱이 김지영의 이름은 이 시대 젊은 여성들의 삶을 보편적으로 그리기 위한 작가의 전략적 선택이기도 하다. 실제로 1982년에 태어난 여아 중 가장 많이 등록된 이름이 ‘지영’이기 때문이다. 김지영이라는 개인의 고백을 30대 여성, 나아가 이 시대 여성들의 고백으로 볼 수 있는 이유다. 

선배는 평소와 똑같이 다정하고 차분히 물었다. 껌이 무슨 잠을 자겠어요, 라고 대답하고 싶었지만 김지영 씨는 입을 다물어 버렸다. (94쪽)

영업 중인 빈 택시 잡아 돈 내고 타면서 고마워하기라도 하라는 건가. 배려라고 생각하며 아무렇지도 않게 무례를 저지르는 사람. 어디서부터 어디까지 항의를 해야 할지도 가늠이 되지 않았고, 괜한 말싸움을 하기도 싫어 김지영 씨는 그냥 눈을 감아 버렸다. (100~101쪽)

주량을 넘어섰다고, 귀갓길이 위험하다고, 이제 그만 마시겠다고 해도 여기 이렇게 남자가 많은데 뭐가 걱정이냐고 반문했다. 니들이 제일 걱정이거든. 김지영 씨는 대답을 속으로 삼키며 눈치껏 빈 컵과 냉면 그릇에 술을 쏟아 버렸다. (116쪽)

조금도 서운하지 않았다. 견딜 수 없는 것은 오히려 그 순간들이었다. 김지영 씨는 충분히 건강하다고, 약 같은 것은 필요 없다고, 가족 계획은 처음 보는 친척들이 아니라 남편과 둘이 하겠다고 말하고 싶었다. 하지만 아니에요, 괜찮아요, 라는 말밖에 할 수 없었다. (133~134쪽)


■기사, 통계, SNS 등 풍부하고 탄탄한 취재
보고서 형식으로 쓰인 『82년생 김지영』의 에피소드들은 무척이나 사실적이다. 어린 시절, 학창 시절, 회사 생활, 결혼 생활에 이르기까지 여성이라면 누구에게나 익숙한 이 경험들은 온라인 커뮤니티와 SNS에서 많은 공감대를 형성한 사례들을 채집한 결과이기 때문이다. 이러한 구체적인 사례들과 함께 등장하는 각종 팩트들은 지난 20여 년 동안의 ‘성차별 역사’를 한눈에 보여 준다. 『확률 가족』 『기록되지 않은 노동』 『고용 동향 브리프』 등의 도서와 「여자라고 전교 회장 못 하나요」 등의 신문 기사를 비롯해 「인구 동태 건수 및 동태율」 「출산 순위별 출생 성비」 같은 통계청 자료, OECD에서 발표한 성별 인금 격차 (Gender wage gap) 자료 및 외신 기사, 「호주제 페지: 호주제, 벽을 넘어 평등 세상으로」 등 행정부 정책 보고서, 「경력단절 여성 지원정책의 현황과 과제」 같은 보건복지포럼 등의 자료가 쉴 새 없이 등장한다. 개인적 기억과 고백의 형식을 취하고 있는 이야기는 이러한 사실적 자료들을 통해 한국에서 살아가는 여성의 보편적인 삶으로 도약하는 근거가 된다. 


추천글 하나도 낯설지가 않은데 새삼 눈물이 고이다니 이상한 일이다. 눈 돌릴 수 없는 통계와 보도 사이, 그리고 눈에 보이지도 않을 만큼 미세한 차별과 폭력 속에 성장한 나와 내 또래 수많은 지영이들의 삶에 대한 담담하고 서글픈 보고서. 어차피 해피엔딩은 오지 않을 것이다. 다만 운 좋게, 혹은 우연히 살아남은 '여아'들이었던 우리는 이렇게 말하고 기록을 남길 수밖에. -최지은(웹진 《아이즈》 기자)"""

In [162]:
tw = Twitter()
query_tokens = tw.morphs(query)

In [159]:
new_vector = model.infer_vector(query_tokens,steps=100) #
sims = model.docvecs.most_similar([new_vector])
sims

[('양유연(Yang Yooyun 楊裕然)', 0.5743376016616821),
 ('최영환(Choi Younghwan)', 0.5583254098892212),
 ('이중섭(Lee Jongsep)', 0.5536295175552368),
 ('최수연', 0.5499486327171326),
 ('피에르 위그 Pierre Huyghe', 0.547799825668335),
 ('신미경', 0.5446125268936157),
 ('엽서', 0.5438199043273926),
 ('하태범(Ha Taebum)', 0.5423631072044373),
 ('신혜선', 0.5391435623168945),
 ('육근병', 0.5384809970855713)]

In [163]:
new_vector = model.infer_vector(query_tokens,steps=100) #
sims = model.docvecs.most_similar([new_vector])
sims

[('김숙현', 0.6045628786087036),
 ('예미(YEMI)', 0.55963534116745),
 ('장석원(Chang Sukwon)', 0.5509284138679504),
 ('김시연(Kim Siyeon)', 0.5491480827331543),
 ('차혜림', 0.5460800528526306),
 ('홍일화(Hong Ilhwa 洪逸和)', 0.5409502983093262),
 ('구민자', 0.5409432649612427),
 ('오석근', 0.5265445709228516),
 ('배준현', 0.5238026976585388),
 ('김홍석(Gim Hongsok)', 0.5234020948410034)]

In [160]:
[' '.join(artist[1]) for artist in artists_twice_tokens if artist[0]=='양유연(Yang Yooyun 楊裕然)']

['누군가 는 우리 가 불신 이 팽배 한 사회 에 살 고 있다 고 말한 다 . 또 누군가 는 우리 가 맹신 이 창궐 한 사회 에 살 고 있다 고 말한 다 . 당신 을 지그시 응시 하는 저 사람 이 벗 인지 적 인지 도무지 결정 하기 가 쉽 지 않다 . 당신 에게 다가서는 여러 손 들 은 당신 을 어루 만지 려는 것 도 같고 당신 을 낚아채려 는 것 도 같다 . 각종 매체 는 누구 든 각광 받는 스타 가 되어 갈채 를 받 을 수 있다 고 떠들어 댄 다 . 그러나 사 나 운 눈초리 를 희 번덕 거리 는 온갖 감시 시스템 은 누구 든 범죄자 가 되어 색출 의 대상 이 될 수 있다 고 으름장 을 놓는 다 . 당신 을 비추는 저 불빛 은 영광 의 스포트라이트 인지 추궁 의 서치라이트 인지 도저히 판단할 수가 없다 . 그렇다고 칠흑 같은 어둠 속 에 잠기 자 니 그 곳 이 안온 한 도피 처 인지 망각 과 낙오 의 심연 인지 아리송 하기 만 하다 . 신자 유주 의적 자본 이 맹위 를 떨 치 는 시대 에 우리 모두 는 쇼 윈도 에 전시된 상품 과도 같다 . 그러나 우리 가 흠모 와 매혹 의 대상 으로 격상 할 지 혐오 와 공포 의 대상 으로 전락 할 지 종잡 을 수가 없다 . 쇼 윈도 의 마네킹 처럼 길함 과 흉함 이 합의 점 을 찾지 못 한 상태 가 오늘날 당신 이 처 한 상태 이다 . 요컨대 이 곳 은 과잉 과 결핍 이 서로 화학 적 으로 뒤 섞여 균형 을 이루는 데 실패한 , 단지 물리 적 으로 뒤 엉 켜 있 을 뿐인 불확실 성의 세계 다 . 이 것 이 양 유연 이 바라본 세상 의 풍경 이다 . 불신 과 맹신 , 벗 과 적 , 빛과 어둠 , 서치라이트 와 스포트라이트 , 매혹 과 혐오 등 양 유연 의 회화 를 가로 지르는 이분법 적 대상 들 은 표면 적 으로는 서로 대립 하는 것 처럼 보이 지만 그 에 못지않 게 어떤 공통점 을 지닌 다 . 그것 은 그 양편 이 모두 극단 의 형태 를 띤다 는 점 이다 . 불신 과 맹신 은 모두 사태 의 진상 으로부터 등 을 돌려 버리

In [164]:
[' '.join(artist[1]) for artist in artists_twice_tokens if artist[0]=='김숙현']

['종로 의 원서동 에 위치한 갤러리 아트 스페이스 에서 두 명의 작가 의 개인 전이 개최 된 다 . 김숙현 1 층 의 도시 는 천태만상 의 하나 이다 . 여행 , 북촌 스캔들 , 달밤 체조 , 윗집 남자 아랫 집 여자 , 휴가 , 사랑 찾기 , 바람 쐬기 등 김숙현 작가 가 바라보는 물질 주의 와 빈부 격차 를 묘사해 내 며 , 그 사회 속 에서 순수한 그 만의 로맨스 를 알 차게 묘사한 다 . 그 의 작업 은 이처럼 이야기 구성 을 하게 만든 다 . 김숙현 작가 의 그림 속 에 있는 방독마스크 들 을 이용한 작가 의 나름 의 대면 처리 로 도 묘사 되 지만 그 동안 해온 작가 의 여러 작업 을 구체 적 으로 집중해서 보면 순박 성 을 엿볼 수 있다 . 또한 어떻 게 사는 게 진정 한 행복 인가 이사회 속 에서 갈망 의 몸부림 그런 질문 을 작가 의 작품 이 펼쳐질 예정 이다 . 박상희 2 층 의 이번 주제 는 세트 인 더 시티 북촌 으로 일상 의 풍경 특히 밤 의 야경 도시 를 화면 에 일상 의 풍경 특히 밤 의 야경 도시 를 화면 에 재현 해 왔 다면 도시 를 세트 로 만들어 다시 화면 으로 옮기는 작업 과정 을 거친 후 캔버스 에 옮겨 넣는 다 . 특히 이번 전시 에서는 북촌 이나 경복궁 등 과거 원래 의 목적 과 다르 게 소비 되는 장소 성 을 강조하며 , 그 장소 가 실제 에 가까운 실재 적 가상 을 보여 주 며 , 다양한 느낌 으로 표현된 다 . 또한 그 장소 가 실제 에 가까운 실재 적 가상 을 보여 준 다고 할 수 있겠 다 . 팝업 북 형태 의 세트 안 에서 도시 의 부산물 들 이나 관련된 그림 들 이 세팅 되어 있고 그 장면 을 촬영 한 후 촬영 된 사진 을 그림 으로 다시 그리는 과정 을 거치는 작가 의 다 채 로운 작품 을 만날 수 있다 . 사랑 을 일 깨우는 봄 을 장식 하는 분홍 빛 의 장미 는 사랑 의 아름다 움 을 매혹 적 으로 보여 주 는 상징 으로 여겨집니 다 . 장미 는 많은 시 와 그림 을 통해 행복한 사랑 , 사랑 의 맹세